In [ ]:
import string
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split
import io
import matplotlib.pyplot as plt
 
from __future__ import print_function
from __future__ import division
import json
import numpy as np
 
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Dropout, Flatten, Lambda, Embedding,LSTM
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.initializers import RandomNormal
 
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
 
import tensorflow as tf
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
from pathlib import Path

path = Path(os.getcwd())
path = path/'gdrive'/'My Drive'
#news=path/'Thesis Data'/'full doc csv'
aa=path/'Thesis Data'/'Our dataset'
#aa2=path/'Thesis Data'/'AA dataset'
#char=path/'Thesis Data'/'char-ulm'
#wiki_data = path/'Thesis Data'/'wiki_data'/'Wiki'

In [ ]:
def load_ag_data(trainfile,testfile): # recieve trainfile and testfile as a parameter
  
  onehotencoder = preprocessing.OneHotEncoder(categories='auto') # define one hot encoder

  ## TRAIN
  df = pd.read_csv(trainfile) # read the train csv file
  df = df.dropna() # this method allows the user to analyze and drop rows/columns with null values in diff ways
  df = df.sample(frac=1).reset_index(drop=True)#shuffle
  x_train = np.array(df['text']).reshape(-1,1) # reshape (-1,1) means row is as you have same row in text and column is 1.Now the text is 2D.
  # 'fit' computes the mean and std dev to be used for later scaling(just computation).'transform' uses a previously computed mean and std dev to autoscale the data(subtract mean from all values and divide it by std dev)  
  y_train = onehotencoder.fit_transform((df['label'].values).reshape(-1,1)).toarray() # using 'label' with one hot encoding,reshape and toarray and predict the value which one store in y_train

  ## TEST
  df = pd.read_csv(testfile)
  df = df.dropna()
  df = df.sample(frac=1).reset_index(drop=True)#shuffle
  x_test = np.array(df['text']).reshape(-1,1)
  y_test = onehotencoder.transform((df['label'].values).reshape(-1,1)).toarray() # toarray return a dense ndarray representation of this matrix

  return (x_train, y_train), (x_test, y_test)

In [ ]:
def create_vocab_set():
    alphabet=" !\"#$%&'()*+,-./0123456789:;=<>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|}~¥§©±ঃঅআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহ়ািীুূৃেৈোৌ্ৎৗড়ঢ়য়০১২৩৪৫৬৭৮৯৷‘’‚“”‪™−√∝∞"    
    vocab_size = len(alphabet)
    print(vocab_size)
    vocab = {}
    reverse_vocab = {}
    for ix, t in enumerate(alphabet):
        vocab[t] = ix # each alphabet stores in 'vocab' dictionary,means index,vocabulary(key-value) pair
        reverse_vocab[ix] = t # each index number stores in 'vocab' as a index,means vocab,index(key-value) pair 
    return vocab, reverse_vocab, vocab_size, alphabet

In [ ]:
def encode_data(x, maxlen, vocab): # x is train data or test data,maxlen is 3000,vocab is dictionary where key-value pair is indices and alphabet,take from function's parameter
    # Iterate over the loaded data and create a matrix of size (len(x), maxlen)
    # Each character is encoded into a one-hot array
    # Chars not in the vocab are encoded as -1, into an all zero vector.
    input_data = np.zeros((len(x), maxlen), dtype=np.int) # create a matrix with zero,row size is len(x) and column size is maxlen
    for dix, sent in enumerate(x): # take the index and text of x(train data/test data)
        counter = 0
        for c in sent[0]: # loop over throuh each word in a sentence
            if counter >= maxlen: # if the counter is greater then maxlen,then pass it,nothing to do
                pass
            else:                  # if the counter is less then maxlen then get the index and update it
                ix = vocab.get(c, -1)  # get index from vocab dictionary, if not in vocab, return -1
                input_data[dix, counter] = ix # take the 'ix'(index) on 'dix' and 'counter' of 'input_data'
                counter += 1 # increase the counter
    return input_data

In [ ]:
def create_model(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter, cat_output, w): # w describe 'weight'
    
    initializer = RandomNormal(mean=0.0, stddev=0.05, seed=None)

    # Define what the input shape looks like
    inputs = Input(shape=(maxlen,), dtype='int64')

    # print(w)
    embedded = Embedding(input_dim=vocab_size, output_dim=vocab_size , weights=[w])(inputs) # multiply 'inputs' with embedding,
    # where 'input_dim' is vocab_size and 'output_dim' is also vocab_size

    # All the convolutional layers...,'nb_filter[0]' is 64,'nb_filter[1]' is 128,'nb_filter[2]' is 256,'nb_filter[3]' is 256
    # and kernel takes 7,then 3,then 1,then 1 (arbitrary)
    conv = Convolution1D(filters=nb_filter[0], kernel_size=filter_kernels[0], kernel_initializer=initializer,
                         padding='valid', activation='relu',
                         input_shape=(maxlen, vocab_size))(embedded) # mult by 'embedded',i/p shape is (maxlen,vocab_size)
    conv = MaxPooling1D(pool_size=3)(conv) # here using (3*3) max pool/avg pool

    conv1 = Convolution1D(filters=nb_filter[1], kernel_size=filter_kernels[1], kernel_initializer=initializer,
                          padding='valid', activation='relu')(conv) # we don't create extra zero padding here
    conv1 = MaxPooling1D(pool_size=3)(conv1)

    conv2 = Convolution1D(filters=nb_filter[2], kernel_size=filter_kernels[2], kernel_initializer=initializer,
                          padding='valid', activation='relu')(conv1)
    conv2 = MaxPooling1D(pool_size=3)(conv2)

    conv3 = Convolution1D(filters=nb_filter[3], kernel_size=filter_kernels[3], kernel_initializer=initializer,
                          padding='valid', activation='relu')(conv2)
    conv3 = MaxPooling1D(pool_size=3)(conv3)
    conv3 = Flatten()(conv3)

    # Two dense layers with dropout of .5
    z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(conv3))

    # Output dense layer with softmax activation
    pred = Dense(cat_output, activation='softmax', name='output')(z) # predicted the o/p using softmax activation

    model = Model(inputs=inputs, outputs=pred) # create a instance of Model where 'inputs' is inputs and outputs is 'pred'

    sgd = SGD(lr=0.01, momentum=0.9) # here create a instance of SGD opt. where learning rate is=0.01 and momentum is=0.9
    adam = Adam(lr=0.001, decay=0.0001) # here create a instance of Adam opt
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy']) # using adam optimizer where 'metrics'
    # is acuracy

    return model


In [ ]:
def CharCNN(trainfile,testfile,aa,cat_output,pre=False,w=0):
  np.random.seed(123)  # for reproducibility
  # Maximum length. Longer gets chopped. Shorter gets padded.
  maxlen = 3000
  # Model params
  # Filters for conv layers
  nb_filter = [64,128,256,256]
  # Number of units in the dense layer
  dense_outputs = 512
  # Conv layer kernel size
  filter_kernels = [7, 3,1,1]
  # Compile/fit params
  batch_size = 128 # take batch_size is 128 and no of epochs is 15
  nb_epoch = 15

  print('Loading data...')
  (xt, yt), (x_test, y_test) = load_ag_data(trainfile,testfile) # call the 'load_ag_data' function and pass these params

  print('Creating vocab...')
  vocab, reverse_vocab, vocab_size, alphabet = create_vocab_set() # call this function and take these values

  if not pre:
    w=np.eye(vocab_size) # if not 'pre' then create np eye matrix(diagonal matrix of vocab_size) which describe weight
  print('Build model...')
  model = create_model(filter_kernels, dense_outputs, maxlen, vocab_size,nb_filter, cat_output,w) # creare a CNN model  
  
  print('Encode data...')
  xt = encode_data(xt, maxlen, vocab) # pass the parms where 'xt' means train data,maxlen is 3000,vocab is alphabet with index
  x_test = encode_data(x_test, maxlen, vocab)
  print(xt[0])

  print('Chars vocab: {}'.format(alphabet))
  print('Chars vocab size: {}'.format(vocab_size))
  print('X_train.shape: {}'.format(xt.shape))
  model.summary()
  print('Fit model...')
  # fit the model with mean and std dev,train data is 80% and test data is 20% with shuffle  
  hist = model.fit(xt, yt, validation_split=0.2 , batch_size=batch_size, epochs=nb_epoch, shuffle=True)

  history = pd.DataFrame(hist.history)
  plt.figure(figsize=(12,12));
  plt.plot(history["loss"],label='loss');
  plt.plot(history["val_loss"], label='val-loss');
  plt.title("Loss with trained word vectors");
  plt.legend();
  # plt.show();
  plt.savefig(aa+'loss.eps')
  
  Y_pred = model.predict(x_test) # predict the o/p using test datset
  y_pred = np.argmax(Y_pred, axis=1) # this function returns of the maximum values are returned along with the specified axis
  y_test = np.argmax(y_test,axis=1)
  # print(y_pred)
  # print(y_test)
  acc = (y_test==y_pred).sum()/len(y_pred) # if the predict is same as test then return 1,sum all and divide by predicted value
  f1 = f1_score(y_test, y_pred, average='macro') # find f1 score
  print("Accuracy=",acc," F1=",f1)
  cm=confusion_matrix(y_test, y_pred) # draw the confusion matrix using originl label and predicted value
  print(cm)

  df_cm = pd.DataFrame(cm, range(cat_output),range(cat_output))
  plt.figure(figsize = (10,7))
  sn.set(font_scale=1.4)#for label size
  sn.heatmap(df_cm, annot=True,annot_kws={"size": 16},fmt='g')# font size
  plt.savefig(aa+'cm.eps')
  
  return model

In [ ]:
w = model.layers[1].get_weights()[0] # dense layer is a subclass of layer and it weights,which is a type of python list has two elements weight of the layer stored at layer.get_weights()[0] and bias is stored at layer.get_weights()[1]
w = np.array([np.array(xi) for xi in w]) # for each elment of w applying np array,then again apply np array on top of it
np.savetxt('embeds_alphanum.txt', w) # save 'w' on this text file
print(w)

NameError: ignored

In [ ]:
# model = CharCNN(news/'full_doc_train.csv',news/'full_doc_test.csv','news',12)

In [ ]:
model = CharCNN(aa/'ulm_train.csv',aa/'ulm_test.csv','aa',6)

Loading data...
Creating vocab...
181
Build model...
Encode data...
[129 144 117 ... 144 127 153]
Chars vocab:  !"#$%&'()*+,-./0123456789:;=<>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|}~¥§©±ঃঅআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহ়ািীুূৃেৈোৌ্ৎৗড়ঢ়য়০১২৩৪৫৬৭৮৯৷‘’‚“”‪™−√∝∞
Chars vocab size: 181
X_train.shape: (1800, 3000)
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 3000)]            0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 3000, 181)         32761     
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 2994, 64)          81152     
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 998, 64)           0         
________________________________________

InvalidArgumentError: ignored